In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/students-dropout/sample_submission.csv
/kaggle/input/students-dropout/validation.csv
/kaggle/input/students-dropout/train.csv
/kaggle/input/students-dropout/test.csv


### Setting up the dfs

In [2]:
# to train:
train_df = pd.read_csv('/kaggle/input/students-dropout/train.csv')
# to test:
validation_df = pd.read_csv('/kaggle/input/students-dropout/validation.csv')
# to predict:
test_df = pd.read_csv('/kaggle/input/students-dropout/test.csv')

sample_df = pd.read_csv('/kaggle/input/students-dropout/sample_submission.csv')

In [3]:
train_df.describe()

,Id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nacionality,Mother's qualification,Father's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
count,2476.000000,2476.000000,2476.000000,2476.000000,2476.000000,2476.000000,2476.000000,2476.000000,2476.000000,2476.000000,...,2476.000000,2476.000000,2476.000000,2476.000000,2476.000000,2476.000000,2476.000000,2476.000000,2476.000000,2476.000000
mean,2211.848142,1.176494,6.865105,1.722940,9.912763,0.885299,2.457997,1.237480,12.121163,16.460420,...,0.125202,0.553716,6.207997,8.068255,4.466074,10.287523,0.142973,11.558522,1.227181,0.049935
std,1279.257993,0.600966,5.261983,1.304837,4.348326,0.318725,3.878489,1.685393,9.019925,11.009475,...,0.651707,1.914945,2.209427,3.972810,2.987121,5.168928,0.709357,2.653639,1.368978,2.270650
min,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.600000,-0.800000,-4.060000
25%,1078.750000,1.000000,1.000000,1.000000,6.000000,1.000000,1.000000,1.000000,2.000000,3.000000,...,0.000000,0.000000,5.000000,6.000000,2.000000,10.833333,0.000000,9.400000,0.300000,-1.700000
50%,2218.000000,1.000000,8.000000,1.000000,10.000000,1.000000,1.000000,1.000000,13.000000,14.000000,...,0.000000,0.000000,6.000000,8.000000,5.000000,12.221825,0.000000,11.100000,1.400000,0.320000
75%,3319.250000,1.000000,12.000000,2.000000,13.000000,1.000000,1.000000,1.000000,22.000000,27.000000,...,0.000000,0.000000,7.000000,10.000000,6.000000,13.369375,0.000000,13.900000,2.600000,1.790000
max,4420.000000,6.000000,18.000000,6.000000,17.000000,1.000000,17.000000,20.000000,28.000000,34.000000,...,12.000000,18.000000,23.000000,27.000000,20.000000,18.571429,12.000000,16.200000,3.700000,3.510000


In [4]:
train_df.isnull().sum()
#train_df.dtypes

Id                                                0
Marital status                                    0
Application mode                                  0
Application order                                 0
Course                                            0
Daytime/evening attendance                        0
Previous qualification                            0
Nacionality                                       0
Mother's qualification                            0
Father's qualification                            0
Mother's occupation                               0
Father's occupation                               0
Displaced                                         0
Educational special needs                         0
Debtor                                            0
Tuition fees up to date                           0
Gender                                            0
Scholarship holder                                0
Age at enrollment                                 0
Internationa

In [5]:
### Understading the target

In [6]:
train_df['Category'].unique()

array(['Graduate', 'Dropout', 'Enrolled'], dtype=object)

### Defining the most impacting featres

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt
#for feature in train_df.columns:
#    print(f'{feature}')
#    sns.displot(data=train_df, x=feature, hue= 'Category', kind="kde")
#    plt.show()

In [8]:
features = ['Application mode',
'Course',
'Scholarship holder',
'Curricular units 1st sem (approved)',
'Curricular units 2nd sem (approved)',
'Curricular units 1st sem (grade)',
'Curricular units 2nd sem (grade)',

'Debtor',
'Curricular units 2nd sem (evaluations)',
'Curricular units 1st sem (evaluations)',
'Tuition fees up to date',
'Gender',
'Age at enrollment',
'Curricular units 1st sem (enrolled)',
'Curricular units 2nd sem (enrolled)',
'Unemployment rate',

'Displaced',
"Mother's qualification",
"Father's qualification",
"Father's occupation",
'Previous qualification',
'Inflation rate',
'GDP']

In [9]:
numerical = [ 'Age at enrollment',
'Unemployment rate',
'Previous qualification',
'Inflation rate',
'GDP',
'Curricular units 1st sem (approved)',
'Curricular units 2nd sem (approved)',
'Curricular units 1st sem (grade)',
'Curricular units 2nd sem (grade)',
'Curricular units 1st sem (evaluations)',
'Curricular units 2nd sem (evaluations)',
'Curricular units 1st sem (enrolled)',
'Curricular units 2nd sem (enrolled)']

In [10]:
"""for feature in numerical:
    print(f'{feature}')
    sns.displot(data=train_df, x=feature, hue= 'Category', kind="kde")
    plt.show()"""

'for feature in numerical:\n    print(f\'{feature}\')\n    sns.displot(data=train_df, x=feature, hue= \'Category\', kind="kde")\n    plt.show()'

In [11]:
bools = ['Scholarship holder',
'Debtor',
'Tuition fees up to date',
'Gender',
'Displaced']

In [12]:
"""
for feature in bools:
    print(f'{feature}')
    sns.boxplot(data=train_df, x=feature,y='Curricular units 2nd sem (approved)', hue= 'Category', showfliers=False)
    plt.show()
    """

"\nfor feature in bools:\n    print(f'{feature}')\n    sns.boxplot(data=train_df, x=feature,y='Curricular units 2nd sem (approved)', hue= 'Category', showfliers=False)\n    plt.show()\n    "

In [13]:
categorical = ['Application mode',
'Course',
"Mother's qualification",
"Father's qualification",
"Father's occupation"]

In [14]:
"""
from matplotlib import rcParams

rcParams['figure.figsize'] = 11.7,8.27

for feature in categorical:
    print(f'{feature}')
    sns.barplot(data=train_df, y=feature,x='Curricular units 2nd sem (approved)', hue= 'Category', orient = 'h')#, showfliers=False )
    plt.show()
    """

"\nfrom matplotlib import rcParams\n\nrcParams['figure.figsize'] = 11.7,8.27\n\nfor feature in categorical:\n    print(f'{feature}')\n    sns.barplot(data=train_df, y=feature,x='Curricular units 2nd sem (approved)', hue= 'Category', orient = 'h')#, showfliers=False )\n    plt.show()\n    "

### Setting up y

In [15]:
y_train = train_df['Category']
y_test = validation_df['Category']

### Setting X with all the features

In [16]:
from sklearn.preprocessing import StandardScaler

#X_test = validation_df[features]
X_train_all = train_df.drop(columns=['Category'])
X_test_all = validation_df.drop(columns=['Category'])

sc = StandardScaler()
X_train_all = sc.fit_transform(X_train_all)
X_test_all = sc.transform(X_test_all)

### Setting X with relevant features

In [17]:
X_train =  train_df[features]
X_test = validation_df[features]
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Setting a df without too related features

In [18]:
df = pd.concat([train_df, validation_df])

clean_df = df.copy()
bad_features = ['International','Curricular units 1st sem (enrolled)','Curricular units 1st sem (credited)' ,'Curricular units 1st sem (approved)'  ]
clean_df = clean_df.drop(columns= bad_features)
print (df.shape, clean_df.shape)

(3804, 36) (3804, 32)


In [19]:
clean_train_df = train_df.drop(columns= bad_features)

clean_validation_df = validation_df.drop(columns= bad_features)


## Implementation of relevant machine learning models.


In [20]:
from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import XGBRFClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


In [21]:
"""
gbc = GradientBoostingClassifier()
xgbc = XGBClassifier()
XGBRFC = XGBRFClassifier()
rfc = RandomForestClassifier()
# setting the cross validation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model 1
n_scores = cross_val_score(gbc, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# evaluate the model 2
n_scores1 = cross_val_score(xgbc, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# evaluate the model 3
n_scores2 = cross_val_score(XGBRFC, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# evaluate the model 4
n_scores3 = cross_val_score(rfc, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

# report performance of the models
print('gbc','Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
print('xgbc','Accuracy: %.3f (%.3f)' % (mean(n_scores1), std(n_scores1)))
print('xgbrf','Accuracy: %.3f (%.3f)' % (mean(n_scores2), std(n_scores2)))
print('rfc','Accuracy: %.3f (%.3f)' % (mean(n_scores3), std(n_scores3)))


"""

"\ngbc = GradientBoostingClassifier()\nxgbc = XGBClassifier()\nXGBRFC = XGBRFClassifier()\nrfc = RandomForestClassifier()\n# setting the cross validation\ncv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)\n# evaluate the model 1\nn_scores = cross_val_score(gbc, X, y, scoring='accuracy', cv=cv, n_jobs=-1)\n# evaluate the model 2\nn_scores1 = cross_val_score(xgbc, X, y, scoring='accuracy', cv=cv, n_jobs=-1)\n# evaluate the model 3\nn_scores2 = cross_val_score(XGBRFC, X, y, scoring='accuracy', cv=cv, n_jobs=-1)\n# evaluate the model 4\nn_scores3 = cross_val_score(rfc, X, y, scoring='accuracy', cv=cv, n_jobs=-1)\n\n# report performance of the models\nprint('gbc','Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))\nprint('xgbc','Accuracy: %.3f (%.3f)' % (mean(n_scores1), std(n_scores1)))\nprint('xgbrf','Accuracy: %.3f (%.3f)' % (mean(n_scores2), std(n_scores2)))\nprint('rfc','Accuracy: %.3f (%.3f)' % (mean(n_scores3), std(n_scores3)))\n\n\n"

gbc Accuracy: 0.772 (0.014)

xgbc Accuracy: 0.772 (0.021)

xgbrf Accuracy: 0.762 (0.015)

rfc Accuracy: 0.771 (0.015)

### Random forest

In [22]:
rfc = RandomForestClassifier()
#clf.fit(X, y)
#RandomForestClassifier(...)
#print(clf.predict([[0, 0, 0, 0]]))
rfc.fit(X_train_all, y_train)
rfc.score(X_test_all, y_test)

0.7582831325301205

### Predicting with the GradientBoostingClassifier

In [23]:
gbc = GradientBoostingClassifier().fit(X_train_all, y_train)
gbc.score(X_test_all, y_test) #0.766566265060241
gbc.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'deviance',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

### Tuning the GradientBoostingClassifier

In [24]:
"""
base_params = {'ccp_alpha': [0.0],
 'criterion':[ 'friedman_mse'],
 'init': [None],
 'learning_rate': [0.1,1],
 'loss': ['deviance'],
 'max_depth': [3,5],
 'max_features': [None, 0.8],
 'max_leaf_nodes': [None], #{'learning_rate': 1, 'max_depth': 5, 'max_features': 0.8, 'n_estimators': 120, 'subsample': 1}
 'min_impurity_decrease':[ 0.0],
 'min_samples_leaf': [1],
 'min_samples_split': [2],
 'min_weight_fraction_leaf':[ 0.0],
 'n_estimators': [100,120],
 'n_iter_no_change': [None],
 'random_state': [None],
 'subsample': [1.0],
 'tol': [0.0001],
 'validation_fraction': [0.1],
 'verbose': [0],
 'warm_start': [False]}
 
params_gbc = {
    'subsample': [1,0.5],
    'n_estimators': [120,150, 170],
    'learning_rate':[1,1.1],
    'max_depth': [5, 3],
    'max_features': [0.4, 0.8]
    #{'learning_rate': 1, 'max_depth': 5, 'max_features': 0.8, 'n_estimators': 120, 'subsample': 1}
    #{'learning_rate': 1, 'n_estimators': 150, 'subsample': 1}
    

}
grid_gbc = GridSearchCV(estimator = GradientBoostingClassifier(), 
                         param_grid=base_params, scoring ='accuracy', 
                         cv=10, n_jobs=-1 )
grid_gbc = grid_gbc.fit(X_train, y_train)
print(grid_gbc.best_params_)
print(grid_gbc.best_score_)
"""

"\nbase_params = {'ccp_alpha': [0.0],\n 'criterion':[ 'friedman_mse'],\n 'init': [None],\n 'learning_rate': [0.1,1],\n 'loss': ['deviance'],\n 'max_depth': [3,5],\n 'max_features': [None, 0.8],\n 'max_leaf_nodes': [None], #{'learning_rate': 1, 'max_depth': 5, 'max_features': 0.8, 'n_estimators': 120, 'subsample': 1}\n 'min_impurity_decrease':[ 0.0],\n 'min_samples_leaf': [1],\n 'min_samples_split': [2],\n 'min_weight_fraction_leaf':[ 0.0],\n 'n_estimators': [100,120],\n 'n_iter_no_change': [None],\n 'random_state': [None],\n 'subsample': [1.0],\n 'tol': [0.0001],\n 'validation_fraction': [0.1],\n 'verbose': [0],\n 'warm_start': [False]}\n \nparams_gbc = {\n    'subsample': [1,0.5],\n    'n_estimators': [120,150, 170],\n    'learning_rate':[1,1.1],\n    'max_depth': [5, 3],\n    'max_features': [0.4, 0.8]\n    #{'learning_rate': 1, 'max_depth': 5, 'max_features': 0.8, 'n_estimators': 120, 'subsample': 1}\n    #{'learning_rate': 1, 'n_estimators': 150, 'subsample': 1}\n    \n\n}\ngrid_gb

In [25]:
gbc = GradientBoostingClassifier(n_estimators=120, learning_rate=0.1,max_depth= 5,
    max_features= 0.8, subsample= 1, random_state=0).fit(X_train_all, y_train)
#{'learning_rate': 1, 'max_depth': 5, 'max_features': 0.8, 'n_estimators': 120, 'subsample': 1}
# 'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 
# 'max_depth': 5, 'max_features': 0.8, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 
# 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 120, 
# 'n_iter_no_change': None, 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 
# 'verbose': 0, 'warm_start': False
gbc.score(X_test_all, y_test)
#0.749
#0.7469879518072289

0.7560240963855421

In [26]:
gbc = GradientBoostingClassifier(n_estimators=120, learning_rate=0.1,max_depth= 5,
    max_features= 0.8, subsample= 1, random_state=0)
X_df = df.drop(columns=['Category'])
y_df = df['Category'].copy()
y_df_num = y_df.replace({'Graduate': 2, 'Enrolled': 1, 'Dropout': 0})

final = gbc.fit(X_df, y_df_num)

In [27]:
#X_target = test_df[features]
y_target = final.predict(test_df)

#y_target = y_train.replace({ 2:'Graduate', 1 : 'Enrolled', 0 : 'Dropout'}, inplace = True)
#print(len(y_target))

#xpected 'Id' column to be of type 'Int32', but was 'String'
#ERROR: Unable to convert 'Id' column row 51 value 'Graduate' to expected type 'Int32'
#ERROR: Unable to convert 'Id' column row 333 value 'Enrolled' to expected type 'Int32'
target_df2 = pd.DataFrame(test_df['Id'])
target_df2['Id']=target_df2['Id'].astype(int)
Category = y_target.tolist()

target_df2['Category'] = Category
target_df2.sort_values(by=['Id'])

,Id,Category
332,1,0
50,2,0
224,9,1
605,18,2
57,22,2
...,...,...
399,4375,0
304,4398,2
157,4404,0
267,4406,1


In [28]:
target_df2['Category'].replace({ 2:'Graduate', 1 : 'Enrolled', 0 : 'Dropout'}, inplace = True)
#print(target_df2.sort_values(by=['Id']))
target_df2.to_csv('submission.csv',index=False)  

target_df3 = pd.read_csv('submission.csv')
target_df3.sort_values(by=['Id'])

,Id,Category
332,1,Dropout
50,2,Dropout
224,9,Enrolled
605,18,Graduate
57,22,Graduate
...,...,...
399,4375,Dropout
304,4398,Graduate
157,4404,Dropout
267,4406,Enrolled


## Model evaluation

In [29]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


### Creating a df merging train and test for fitting 

In [30]:
# evaluate xgboost algorithm for classification

clean_df['Category'].replace({'Graduate': 2, 'Enrolled': 1, 'Dropout': 0},inplace = True)
y = clean_df['Category']
y.value_counts()
sc = StandardScaler()
# X = sc.fit_transform(X)

### K Fold cross validation

In [31]:
gbc = GradientBoostingClassifier(n_estimators=150, learning_rate=1.0,max_depth= 5,
    max_features= 0.4, subsample= 1, random_state=0).fit(X_train_all, y_train)
kfold_validation = KFold(10)

In [32]:
"""results = cross_val_score(gbc, X,y, cv=kfold_validation)
print(results)
print(np.mean(results))"""

'results = cross_val_score(gbc, X,y, cv=kfold_validation)\nprint(results)\nprint(np.mean(results))'

## Submitting the first results


In [33]:
train_df['Category'].unique()

array(['Graduate', 'Dropout', 'Enrolled'], dtype=object)

In [34]:
X_target_gbc = test_df
y_target_gbc = gbc.predict(test_df)
target_df = pd.DataFrame(test_df['Id'])
Category = y_target_gbc.tolist()
target_df['Category'] = Category

#target_df.to_csv('submission.csv',index=False)  

#target_df = pd.read_csv('submission.csv')
#target_df['Category'].unique()

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [35]:
#from sklearn.model_selection import RepeatedKFold

#rkf = RepeatedKFold(n_splits=2, n_repeats=2, random_state=2652124)
#for train_index, test_index in rkf.split(X):
# define model evaluation method
#cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
#scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

### Setting a numeric y

In [36]:
y_train_num = y_train.replace({'Graduate': 2, 'Enrolled': 1, 'Dropout': 0})
y_test_num = y_test.replace({'Graduate': 2, 'Enrolled': 1, 'Dropout': 0})

### Esploring the XGBRFClassifier

In [37]:
XGBRFC = XGBRFClassifier( n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

xgb = XGBRFC.fit(X_train_all, y_train_num)
xgb.score(X_test_all, y_test_num)

0.7612951807228916

In [38]:
XGBRFClassifier().get_params().keys()

dict_keys(['colsample_bynode', 'learning_rate', 'reg_lambda', 'subsample', 'objective', 'use_label_encoder', 'base_score', 'booster', 'callbacks', 'colsample_bylevel', 'colsample_bytree', 'early_stopping_rounds', 'enable_categorical', 'eval_metric', 'gamma', 'gpu_id', 'grow_policy', 'importance_type', 'interaction_constraints', 'max_bin', 'max_cat_to_onehot', 'max_delta_step', 'max_depth', 'max_leaves', 'min_child_weight', 'missing', 'monotone_constraints', 'n_estimators', 'n_jobs', 'num_parallel_tree', 'predictor', 'random_state', 'reg_alpha', 'sampling_method', 'scale_pos_weight', 'tree_method', 'validate_parameters', 'verbosity'])

### Tuning the XGBRFClassifier

In [39]:
"""
from sklearn.model_selection import GridSearchCV
params_xgb = {
    'n_estimators':[600, 500,300],
    'max_depth':[11, 10],
    'eta':[0.1, 0.15],
    'subsample':[0.9,0.8], 
    'colsample_bytree':[0.5, 0.6]
} #{'colsample_bytree': 0.6, 'eta': 0.1, 'max_depth': 10, 'n_estimators': 500, 'subsample': 0.8}
#{'colsample_bytree': 0.6, 'eta': 0.1, 'max_depth': 11, 'n_estimators': 500, 'subsample': 0.9}


grid_xgb = GridSearchCV(estimator = XGBRFClassifier(), 
                         param_grid=params_xgb, scoring ='accuracy', 
                         cv=10, n_jobs=-1 )
grid_xgb.fit(X_train, y_train_num)
print(grid_xgb.best_params_)
"""

"\nfrom sklearn.model_selection import GridSearchCV\nparams_xgb = {\n    'n_estimators':[600, 500,300],\n    'max_depth':[11, 10],\n    'eta':[0.1, 0.15],\n    'subsample':[0.9,0.8], \n    'colsample_bytree':[0.5, 0.6]\n} #{'colsample_bytree': 0.6, 'eta': 0.1, 'max_depth': 10, 'n_estimators': 500, 'subsample': 0.8}\n#{'colsample_bytree': 0.6, 'eta': 0.1, 'max_depth': 11, 'n_estimators': 500, 'subsample': 0.9}\n\n\ngrid_xgb = GridSearchCV(estimator = XGBRFClassifier(), \n                         param_grid=params_xgb, scoring ='accuracy', \n                         cv=10, n_jobs=-1 )\ngrid_xgb.fit(X_train, y_train_num)\nprint(grid_xgb.best_params_)\n"

In [40]:
XGBRFC = XGBRFClassifier( #n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
    colsample_bytree= 0.6, 
    eta= 0.1, 
    max_depth= 11, 
    n_estimators= 500, 
    subsample= 0.9)
#{'colsample_bytree': 0.6, 'eta': 0.1, 'max_depth': 10, 'n_estimators': 500, 'subsample': 0.8}
#{'colsample_bytree': 0.6, 'eta': 0.1, 'max_depth': 11, 'n_estimators': 500, 'subsample': 0.9}

xgb = XGBRFC.fit(X_train_all, y_train_num)
xgb.score(X_test_all, y_test_num)

0.7658132530120482

In [41]:
#X_target = test_df[features]
X_target_all = test_df.copy()
y_target_num = xgb.predict(X_target_all)

#y_target = y_train.replace({ 2:'Graduate', 1 : 'Enrolled', 0 : 'Dropout'}, inplace = True)
#print(len(y_target))
target_df2 = pd.DataFrame(test_df['Id'])

Category = y_target_num.tolist()

target_df2['Category'] = Category
target_df2

,Id,Category
0,3557,0
1,1980,2
2,1112,2
3,2914,0
4,2745,2
...,...,...
615,335,2
616,1639,2
617,1381,0
618,2455,2


In [42]:
"""target_df2.replace({ 2:'Graduate', 1 : 'Enrolled', 0 : 'Dropout'}, inplace = True)
print(target_df2)"""

"target_df2.replace({ 2:'Graduate', 1 : 'Enrolled', 0 : 'Dropout'}, inplace = True)\nprint(target_df2)"

In [43]:
"""target_df2.to_csv('submission.csv',index=False)  

target_df2 = pd.read_csv('submission.csv')
target_df2"""

"target_df2.to_csv('submission.csv',index=False)  \n\ntarget_df2 = pd.read_csv('submission.csv')\ntarget_df2"

### Esploring the XGBClassifier

In [44]:
xgbc = XGBClassifier( n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

xgbc = xgbc.fit(X_train_all, y_train_num)
xgbc.score(X_test_all, y_test_num)
xgbc.get_params()

{'objective': 'multi:softprob',
 'use_label_encoder': False,
 'base_score': 0.5,
 'booster': 'gbtree',
 'callbacks': None,
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 0.8,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'gamma': 0,
 'gpu_id': -1,
 'grow_policy': 'depthwise',
 'importance_type': None,
 'interaction_constraints': '',
 'learning_rate': 0.100000001,
 'max_bin': 256,
 'max_cat_to_onehot': 4,
 'max_delta_step': 0,
 'max_depth': 7,
 'max_leaves': 0,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 1000,
 'n_jobs': 0,
 'num_parallel_tree': 1,
 'predictor': 'auto',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'sampling_method': 'uniform',
 'scale_pos_weight': None,
 'subsample': 0.7,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': None,
 'eta': 0.1}

### Tuning the XGBClassifier

In [45]:
"""
base_params_xgb = {'objective': ['multi:softprob'],
 'use_label_encoder': [False],
 'base_score': [0.5],
 'booster': ['gbtree'],
 'callbacks': [None],
 'colsample_bylevel': [1],
 'colsample_bynode': [1],
 'colsample_bytree': [0.8],
 'early_stopping_rounds': [None],
 'enable_categorical': [False],
 'eval_metric': [None],
 'gamma':[0, 0.1, 0.5, 0.8],
 'gpu_id': [-1],
 'grow_policy': ['depthwise'],
 'importance_type': [None],
 'interaction_constraints': [''],
 'learning_rate': [0.100000001],# {'colsample_bytree': 0.8, 'eta': 0.1, 'max_depth': 6, 'n_estimators': 1000, 'subsample': 0.6, 'tree_method': 'hist'}
 'max_bin': [256],
 'max_cat_to_onehot':[4],
 'max_delta_step': [0],
 'max_depth': [7,6],
 'max_leaves': [0],
 'min_child_weight': [1],
 #'missing': [nan],
 'monotone_constraints': ['()'],
 'n_estimators': [1000],
 'n_jobs': [0],
 'num_parallel_tree': [1],
 'predictor': ['auto'],
 'random_state': [0],
 'reg_alpha': [0],
 'reg_lambda': [1],
 'sampling_method': ['uniform'],
 'scale_pos_weight': [None],
 'subsample': [0.7,0.6],
 'tree_method': ['exact','hist'],
 'validate_parameters': [1],
 'verbosity': [None],
 'eta': [0.1]}
params_xgb = {
    'tree_method':['hist'],
    'n_estimators':[1000,1100],
    'max_depth':[6, 7],#max_depth, min_child_weight and gamma
    'eta':[0.1,0.15], #reduce stepsize eta. Remember to increase num_round when you do so.
    'subsample':[0.6,0.66], # subsample and colsample_bytree.
    'colsample_bytree':[0.8, 0.9]
}#n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8
# {'colsample_bytree': 0.8, 'eta': 0.1, 'max_depth': 6, 'n_estimators': 1000, 'subsample': 0.6, 'tree_method': 'hist'}


grid_xgb = GridSearchCV(estimator = XGBClassifier(), 
                         param_grid=base_params_xgb, scoring ='accuracy', 
                         cv=10, n_jobs=-1 )
grid_xgb.fit(X_train, y_train_num)
print(grid_xgb.best_params_)
"""

"\nbase_params_xgb = {'objective': ['multi:softprob'],\n 'use_label_encoder': [False],\n 'base_score': [0.5],\n 'booster': ['gbtree'],\n 'callbacks': [None],\n 'colsample_bylevel': [1],\n 'colsample_bynode': [1],\n 'colsample_bytree': [0.8],\n 'early_stopping_rounds': [None],\n 'enable_categorical': [False],\n 'eval_metric': [None],\n 'gamma':[0, 0.1, 0.5, 0.8],\n 'gpu_id': [-1],\n 'grow_policy': ['depthwise'],\n 'importance_type': [None],\n 'interaction_constraints': [''],\n 'learning_rate': [0.100000001],# {'colsample_bytree': 0.8, 'eta': 0.1, 'max_depth': 6, 'n_estimators': 1000, 'subsample': 0.6, 'tree_method': 'hist'}\n 'max_bin': [256],\n 'max_cat_to_onehot':[4],\n 'max_delta_step': [0],\n 'max_depth': [7,6],\n 'max_leaves': [0],\n 'min_child_weight': [1],\n #'missing': [nan],\n 'monotone_constraints': ['()'],\n 'n_estimators': [1000],\n 'n_jobs': [0],\n 'num_parallel_tree': [1],\n 'predictor': ['auto'],\n 'random_state': [0],\n 'reg_alpha': [0],\n 'reg_lambda': [1],\n 'sampling_

In [46]:
#ROC curve https://www.youtube.com/watch?v=3fzYdnuvEfk

In [47]:
xgbc = XGBClassifier( tree_method='hist', n_estimators=1000, max_depth=6, eta=0.1, subsample=0.66, colsample_bytree=0.8)
#n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8 -> 0.7635542168674698
#tree_method='hist', n_estimators=1000, max_depth=7, eta=0.1, subsample=0.66, colsample_bytree=0.8 -> 0.7658132530120482
# {'colsample_bytree': 0.8, 'eta': 0.1, 'max_depth': 6, 'n_estimators': 1000, 'subsample': 0.6, 'tree_method': 'hist'} 0.7635542168674698

xgbc = xgbc.fit(X_train_all, y_train_num)
xgbc.score(X_test_all, y_test_num)

0.7605421686746988

In [48]:
corr_features = ['Curricular units 2nd sem (approved)','Curricular units 2nd sem (grade)',
'Curricular units 1st sem (grade)','Tuition fees up to date',
'Scholarship holder','Curricular units 2nd sem (enrolled)']
X_train_corr = clean_train_df.drop(columns=['Category'])
X_test_corr = clean_validation_df.drop(columns=['Category'])

### Submitting the best scoring prediction

In [49]:
xgbc = XGBClassifier( tree_method='hist', n_estimators=1000, max_depth=6, eta=0.1, subsample=0.66, colsample_bytree=0.8)

X_df = df.drop(columns=['Category'])
y_df = df['Category'].copy()
y_df_num = y_df.replace({'Graduate': 2, 'Enrolled': 1, 'Dropout': 0})

final = xgbc.fit(X_df, y_df_num)

In [50]:
#X_target = test_df[features]
y_target = final.predict(test_df)

#y_target = y_train.replace({ 2:'Graduate', 1 : 'Enrolled', 0 : 'Dropout'}, inplace = True)
#print(len(y_target))

#xpected 'Id' column to be of type 'Int32', but was 'String'
#ERROR: Unable to convert 'Id' column row 51 value 'Graduate' to expected type 'Int32'
#ERROR: Unable to convert 'Id' column row 333 value 'Enrolled' to expected type 'Int32'
target_df2 = pd.DataFrame(test_df['Id'])
target_df2['Id']=target_df2['Id'].astype(int)
Category = y_target.tolist()

target_df2['Category'] = Category
target_df2.sort_values(by=['Id'])

,Id,Category
332,1,2
50,2,0
224,9,1
605,18,1
57,22,2
...,...,...
399,4375,0
304,4398,2
157,4404,0
267,4406,1


In [51]:
target_df2['Category'].replace({ 2:'Graduate', 1 : 'Enrolled', 0 : 'Dropout'}, inplace = True)
#print(target_df2.sort_values(by=['Id']))
target_df2.to_csv('submission2.csv',index=False)  

target_df3 = pd.read_csv('submission2.csv')
target_df3.sort_values(by=['Id'])

,Id,Category
332,1,Graduate
50,2,Dropout
224,9,Enrolled
605,18,Enrolled
57,22,Graduate
...,...,...
399,4375,Dropout
304,4398,Graduate
157,4404,Dropout
267,4406,Enrolled
